In [1]:
# Imports

# Functions
from portfolio_analysis import portfolio_returns
from portfolio_analysis import perform_portfolio_analysis

# Libraries
# Plotly for Data Visualization
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')


In [2]:
def plot_portfolio_returns(portfolio_results : dict, analysis_results : dict) -> None:
    """ 
    Portfolio results contains:
        "weights" (ticker_weights), 
        "data" (dataframe), 
        "portfolio returns" (returns)

    Analysis results contains:
        "individual cumulative returns" (individual_cumsum),
        "individual volatility" (individual_vol),
        "individual Sharpe ratio"  (individual_sharpe),
    """

    tickers_weights = portfolio_results["weights"]

    # Pie plot
    fig = go.Figure(data=[go.Pie(
        labels=list(tickers_weights.keys()),    # Obtain tickers 
        values=list(tickers_weights.values()),  # Obtain weights
        hoverinfo='label+percent', 
        textinfo='label+percent',
        hole=.65,
        marker=dict(colors=px.colors.qualitative.G10)
    )])

    # Define layout
    fig.update_layout(title={'text': '<b>Portfolio Allocation</b>'}, height=550)

    # Display Portfolio Allocation plot
    fig.show() 

    # Create subplots for comparison across securities
    fig1 = make_subplots(rows = 1, cols = 2, horizontal_spacing=0.2,
                    column_titles=['Historical Performance Assets', 'Risk-Reward'],
                    column_widths=[.50, .50],
                    shared_xaxes=False, shared_yaxes=False)

    individual_cumsum = analysis_results["individual cumulative returns"]
    individual_vol = analysis_results["individual volatility"]
    individual_sharpe = analysis_results["individual Sharpe ratio"]

    for ticker in individual_cumsum.columns:
        fig1.add_trace(go.Scatter(x=individual_cumsum.index,
                                  y=individual_cumsum[ticker],
                                  mode = 'lines',
                                  name = ticker,
                                  hovertemplate = '%{y:.2f}%',
                                  showlegend=False),
                            row=1, col=1)

    # Define colors for markers on the second subplot
    sharpe_colors = [individual_sharpe[ticker] for ticker in individual_cumsum.columns]

    # Add markers for each ticker on the second subplot
    fig1.add_trace(go.Scatter(x=individual_vol.tolist(),
                              y=individual_cumsum.iloc[-1].tolist(),
                              mode='markers+text',
                              marker=dict(size=75, color = sharpe_colors, 
                                          colorscale = 'Bluered_r',
                                          colorbar=dict(title='Sharpe Ratio'),
                                          showscale=True),
                              name = 'Returns',
                              text = individual_cumsum.columns.tolist(),
                              textfont=dict(color='white'),
                              showlegend=False,
                              hovertemplate = '%{y:.2f}%<br>Annualized Volatility: %{x:.2f}%<br>Sharpe Ratio: %{marker.color:.2f}',
                              textposition='middle center'),
                        row=1, col=2)
            
    # Update layout
    fig1.update_layout(title={'text': f'<b>Portfolio Analysis</b>'},
                       template = 'plotly_white',
                       height = 700, width = 1000,
                       hovermode = 'x unified')
        
    fig1.update_yaxes(title_text='Returns (%)', col=1)
    fig1.update_yaxes(title_text='Returns (%)', col = 2)
    fig1.update_xaxes(title_text = 'Date', col = 1)
    fig1.update_xaxes(title_text = 'Annualized Volatility (%)', col =2)

    # Display plots
    fig1.show()
    
    return

In [ ]:
# Setup inputs
tickers = {
    'SAB.MC' : (1000.0),
    'FER.MC' : (750.0),
    'ITX.MC' : (250.0),
    'MEL.MC' : (2000.0)
}

start: str = '2020-02-18'
end: str = '2025-02-18'
risk_free_rate: float = 0.0125

portfolio_results = portfolio_returns(tickers, start, end)

analysis_results = perform_portfolio_analysis(portfolio_results["data"], portfolio_results["weights"], risk_free_rate)

[*********************100%***********************]  4 of 4 completed

4 Failed downloads:
- SAB.MC: No data found for this date range, symbol may be delisted
- FER.MC: No data found for this date range, symbol may be delisted
- ITX.MC: No data found for this date range, symbol may be delisted
- MEL.MC: No data found for this date range, symbol may be delisted


ValueError: No data available for the following tickers starting from 2024-02-18: SAB.MC, FER.MC, ITX.MC, MEL.MC

In [22]:
plot_portfolio_returns(portfolio_results, analysis_results)